In [3]:
!pip install torch torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 137.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 80.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.transforms import NormalizeFeatures

In [5]:
dataset = Planetoid(root='data/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [6]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
model = GCN(dataset.num_node_features, 32, dataset.num_classes).to(device)

In [9]:
data = data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

In [10]:

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

In [11]:
def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        correct = (pred[mask] == data.y[mask]).sum()
        accs.append(int(correct) / int(mask.sum()))
    return accs

for epoch in range(300):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 5 == 0:
        print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}")

Epoch 000, Loss: 1.9449, Train Acc: 0.3071, Val Acc: 0.2460, Test Acc: 0.2580
Epoch 005, Loss: 1.9379, Train Acc: 0.7357, Val Acc: 0.4820, Test Acc: 0.5090
Epoch 010, Loss: 1.9289, Train Acc: 0.8643, Val Acc: 0.5920, Test Acc: 0.5950
Epoch 015, Loss: 1.9187, Train Acc: 0.8571, Val Acc: 0.6140, Test Acc: 0.5920
Epoch 020, Loss: 1.9076, Train Acc: 0.8929, Val Acc: 0.6280, Test Acc: 0.5970
Epoch 025, Loss: 1.8957, Train Acc: 0.9000, Val Acc: 0.6200, Test Acc: 0.6060
Epoch 030, Loss: 1.8831, Train Acc: 0.9143, Val Acc: 0.6400, Test Acc: 0.6320
Epoch 035, Loss: 1.8697, Train Acc: 0.9143, Val Acc: 0.6580, Test Acc: 0.6520
Epoch 040, Loss: 1.8557, Train Acc: 0.9143, Val Acc: 0.6620, Test Acc: 0.6620
Epoch 045, Loss: 1.8411, Train Acc: 0.9143, Val Acc: 0.6680, Test Acc: 0.6790
Epoch 050, Loss: 1.8259, Train Acc: 0.9143, Val Acc: 0.6800, Test Acc: 0.6900
Epoch 055, Loss: 1.8101, Train Acc: 0.9286, Val Acc: 0.6860, Test Acc: 0.6950
Epoch 060, Loss: 1.7938, Train Acc: 0.9286, Val Acc: 0.6900, Tes

In [12]:
from torch_geometric.loader import ClusterData, ClusterLoader
from torch_geometric.nn import SAGEConv

In [13]:
from torch_geometric.data import Data

In [15]:
edge_index = data.edge_index
x = data.x
y = data.y
train_mask = data.train_mask
val_mask = data.val_mask
test_mask = data.test_mask

In [ ]:
new_data=Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)

In [ ]:
cluster_data = ClusterData(new_data, num_parts=500, recursive=False)

In [16]:
dataset = Planetoid(root='/content/data/Cora', name='Cora')
data = dataset[0]

train_loader = ClusterLoader(data, batch_size=20, shuffle=True, num_workers=4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [32]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn.models import GraphSAGE
from torch_geometric.loader import NeighborLoader
model = GraphSAGE(
    in_channels=dataset.num_node_features,
    hidden_channels=64,
    num_layers=2,
    out_channels=dataset.num_classes,
    dropout=0.5
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
loss_fn = torch.nn.CrossEntropyLoss()

def accuracy(logits, labels):
    preds = logits.argmax(dim=1)
    return (preds == labels).sum().item() / labels.size(0)


for epoch in range(300):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss =  criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        train_acc = accuracy(out[data.train_mask], data.y[data.train_mask])
        val_acc = accuracy(out[data.val_mask], data.y[data.val_mask])
        test_acc = accuracy(out[data.test_mask], data.y[data.test_mask])
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | "
              f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Test Acc: {test_acc:.4f}")

Epoch 000 | Loss: 1.9542 | Train Acc: 0.8429 | Val Acc: 0.3900 | Test Acc: 0.4320
Epoch 001 | Loss: 1.6604 | Train Acc: 0.9500 | Val Acc: 0.6480 | Test Acc: 0.6760
Epoch 010 | Loss: 0.0158 | Train Acc: 1.0000 | Val Acc: 0.7700 | Test Acc: 0.7790
Epoch 020 | Loss: 0.0028 | Train Acc: 1.0000 | Val Acc: 0.7600 | Test Acc: 0.7730
Epoch 030 | Loss: 0.0009 | Train Acc: 1.0000 | Val Acc: 0.7480 | Test Acc: 0.7730
Epoch 040 | Loss: 0.0007 | Train Acc: 1.0000 | Val Acc: 0.7600 | Test Acc: 0.7840
Epoch 050 | Loss: 0.0017 | Train Acc: 1.0000 | Val Acc: 0.7640 | Test Acc: 0.7770
Epoch 060 | Loss: 0.0051 | Train Acc: 1.0000 | Val Acc: 0.7780 | Test Acc: 0.8040
Epoch 070 | Loss: 0.0051 | Train Acc: 1.0000 | Val Acc: 0.7580 | Test Acc: 0.7890
Epoch 080 | Loss: 0.0061 | Train Acc: 1.0000 | Val Acc: 0.7600 | Test Acc: 0.7950
Epoch 090 | Loss: 0.0033 | Train Acc: 1.0000 | Val Acc: 0.7760 | Test Acc: 0.8130
Epoch 100 | Loss: 0.0055 | Train Acc: 1.0000 | Val Acc: 0.7780 | Test Acc: 0.8030
Epoch 110 | Loss